In [75]:
import torch
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print(' Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print(' Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

device: cuda
Tesla P100-PCIE-16GB
 Allocated: 1.0 GB
 Cached:    1.7 GB


In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import math
import numpy as np
import time
import json
import os
if not os.path.exists('models'):
    os.mkdir('models')

torch.manual_seed(3)
plt.style.use('dark_background')
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.rainbow(np.linspace(0, 1, 5)))

N = 3983
A0 = torch.load('../input/am4-default/A-V4_3.pt', map_location='cuda')
capital_ids = torch.load('../input/am4-default/capital_ids.pt', map_location='cuda')
As = torch.stack([A0, A0])
D = torch.load('../input/am4-default/D.pt', map_location='cuda')

mask = ~torch.eye(N, dtype=bool).cuda()
ys = D[mask].reshape(-1)

mask = mask.view(-1)
i = torch.arange(N, device='cuda').repeat(N)[mask]
j = torch.arange(N, device='cuda').repeat_interleave(N)[mask]

xs = torch.stack((i, j), dim=1) # [[i0, j0], ...]

In [77]:
class DemandModel(nn.Module):
    def __init__(self):
        super(DemandModel, self).__init__()
        self.As = nn.Parameter(As)
        self.Bs = nn.Parameter(torch.rand(2, N) * 50)

    def forward(self, batch): # [[a, b], [c, d]], OD pairs
        row_idxs = torch.any(torch.isin(batch, capital_ids), dim=1).long()
        # [any(a in cid, b in cid), any(c in cid, d in did)].int() -> [1, 0]
        A = self.As[row_idxs]
        B = self.Bs[row_idxs]
        
        i, j = batch.t().unsqueeze(2) # i: [[i0], [i1], ...]
        Ai = A.gather(1, i)
        Aj = A.gather(1, j)
        Bi = B.gather(1, i)
        Bj = B.gather(1, j)
        return (Ai * Aj + Bi + Bj).squeeze(1)

model = DemandModel().cuda()

batch_size = N // 2
lr = 0.01 / 2 / 2
status_every = 200#N*(N-1) // (batch_size ) / 1000
epochs = 100

print(f'*** STARTING: {batch_size}, {lr}, {epochs}')
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

dataloader = DataLoader(
    TensorDataset(xs, ys),
    batch_size=batch_size,
    shuffle=True
)

start = time.time()
losses = []
last_model = model
for e in range(epochs):
    total_loss = 0
    i = 0
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        y_pred = model(batch_x)
        loss = criterion(y_pred, batch_y)
        l1 = .002 * sum([p.abs().sum() for p in model.parameters()])
        loss.backward()
        optimizer.step()
        if i > 0 and i % status_every == 0:
            print(f'{i/len(dataloader)+e:.5f},{math.sqrt(loss)},{l1}')
        
        total_loss += loss.item()
        i += 1

    loss = math.sqrt(total_loss / len(dataloader))
#     print(f'{e:>5} | loss={loss}')

    torch.save(model.As, f'./models/As-V1_{e}.pt')
    torch.save(model.Bs, f'./models/Bs-V1_{e}.pt')
#     torch.save(model.C, f'./models/C-V13_{e}.pt')
#     torch.save(model.K, f'./models/M-V12_{e}.pt')

    if loss < 5e-4:
        print(f'*** PAST CRITERON: loss={loss}')
        last_model = model
        break
    if loss < 1 and loss > losses[-1]:
        print(f'*** INCREASING: loss={loss}')
        break
    losses.append(loss)
    last_model = model

print(f'*** DONE: {time.time() - start:.2f}s')
# D_pred = torch.mul(last_model.A.unsqueeze(1), last_model.B) + torch.mul(last_model.A, last_model.B.unsqueeze(1))
# print(D - D_pred)
# print(torch.sum(torch.abs(D - D_pred)))

# plt.plot(losses, label=f'batch_size={batch_size}, lr={lr}')

# plt.legend()
# plt.xlabel('Epoch')
# plt.ylabel('MSE Loss')
# plt.show()

*** STARTING: 1991, 0.0025, 100
0.02511,180.00690633452095,270.0710754394531
0.05021,168.86195360412006,275.818603515625
0.07532,161.28306347071907,281.3601379394531
0.10043,166.84646291575976,286.6864013671875
0.12553,169.05132818910357,291.83251953125
0.15064,159.13345053169996,296.7510681152344
0.17575,157.39209473278828,301.4444885253906
0.20085,151.25107825028886,305.945068359375
0.22596,149.44372765149762,310.22015380859375
0.25107,142.30235745587632,314.27740478515625
0.27617,145.43651294250355,318.1256408691406
0.30128,136.80506204084702,321.7480163574219
0.32639,143.5071249864445,325.1703796386719
0.35149,141.25305582977666,328.3782043457031
0.37660,142.4016862062209,331.37945556640625
0.40171,126.80206493537477,334.2259216308594
0.42681,122.03404686295542,336.89312744140625
0.45192,122.82018105751187,339.4010925292969
0.47703,124.7131865772421,341.7319641113281
0.50213,116.90310908638187,343.9193420410156
0.52724,129.084335319976,345.9983215332031
0.55235,114.82127756321997,3

KeyboardInterrupt: 

In [89]:
print(torch.mean(model.As[0]))
print(torch.mean(model.As[1]))
print(torch.mean(model.Bs[0]))
print(torch.mean(model.Bs[1]))

tensor(15.1053, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(16.6718, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(7.1030, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(10.1553, device='cuda:0', grad_fn=<MeanBackward0>)


In [91]:
torch.save(model.As, 'As_V1.pt')
torch.save(model.Bs, 'Bs_V1.pt')

In [ ]:
os.listdir('./models')
import shutil
shutil.copy('./models/A-V7_3.pt', './A-V7_3.pt')
shutil.copy('./models/M-V7_3.pt', './M-V7_3.pt')

# import shutil
# shutil.rmtree('models')

In [ ]:
for f in os.listdir('./models'):
    os.remove(f)
